In [1]:
import os
os.chdir("../")

In [2]:
os.getcwd()

'/home/msds2025/acastillo/dmw2-2025/lab1/dmw2'

In [14]:
import ast
import re

import pandas as pd
import numpy as np

from dateutil import parser
from collections import Counter
from datetime import datetime
from utils.utils import extract_isbn_details

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
df = pd.read_csv("data/processed/reduced_editions_after_2015.csv")
df.dropna(subset=["title", "publishers"], inplace=True)
df.head()

,record_type,book_id,publishers,title,description,first_sentence,subjects,date_created,isbn_13,publish_date,publish_country,number_of_pages,latest_revision,work_id
2,/type/edition,/books/OL26474165M,['Harlequin Books'],The Kentucky cowboy's baby,Pepper Bourne has big plans for her stepdaddy'...,NaN,"['Fiction', 'Ranches', 'Man-woman relationship...",2018-07-22T05:36:52.580542,['9780373757244'],2016,onc,216.0,3,/works/OL17897021W
4,/type/edition,/books/OL26869357M,['Si chuan wen yi chu ban she'],Bai ma ke xin de xing kong,NaN,NaN,"['Zhang pian xiao shuo', 'Er tong wen xue']",2019-04-28T07:08:59.763359,['9787541143113'],2016,cc,259.0,1,/works/OL19649589W
19,/type/edition,/books/OL26926535M,['U Media Global'],Bu dong sa jiao de nü ren,NaN,NaN,"['Television mini-series', 'Chinese Television...",2019-05-23T21:39:27.766919,000,2017,cau,1260.0,1,/works/OL19713398W
23,/type/edition,/books/OL26927268M,['Owlkids Books'],When planet Earth was new,The story of planet Earth s evolution: 4.5 bil...,NaN,"['Origin', 'Life', 'Natural history', 'Evoluti...",2019-05-23T21:54:23.974566,['9781771472036'],2017,onc,NaN,3,/works/OL19714130W
25,/type/edition,/books/OL26927601M,['Huang guan wen hua chu ban you xian gong si'],"Ren sheng a, zhen de shi mei xiang dao",NaN,NaN,"['Chinese essays', 'Anecdotes']",2019-05-23T22:01:03.251239,['9789573333043'],2017,ch,223.0,1,/works/OL19714463W


In [5]:
df.isna().sum()

record_type              0
book_id                  0
publishers               0
title                    0
description        1208115
first_sentence     1229038
subjects                 0
date_created             0
isbn_13                  0
publish_date             0
publish_country    1197988
number_of_pages     788961
latest_revision          0
work_id                  0
dtype: int64

In [6]:
df["publishers"] = df["publishers"].astype(str).apply(lambda x: ast.literal_eval(x)[0] if x.startswith('[') else x)
df.head()

,record_type,book_id,publishers,title,description,first_sentence,subjects,date_created,isbn_13,publish_date,publish_country,number_of_pages,latest_revision,work_id
2,/type/edition,/books/OL26474165M,Harlequin Books,The Kentucky cowboy's baby,Pepper Bourne has big plans for her stepdaddy'...,NaN,"['Fiction', 'Ranches', 'Man-woman relationship...",2018-07-22T05:36:52.580542,['9780373757244'],2016,onc,216.0,3,/works/OL17897021W
4,/type/edition,/books/OL26869357M,Si chuan wen yi chu ban she,Bai ma ke xin de xing kong,NaN,NaN,"['Zhang pian xiao shuo', 'Er tong wen xue']",2019-04-28T07:08:59.763359,['9787541143113'],2016,cc,259.0,1,/works/OL19649589W
19,/type/edition,/books/OL26926535M,U Media Global,Bu dong sa jiao de nü ren,NaN,NaN,"['Television mini-series', 'Chinese Television...",2019-05-23T21:39:27.766919,000,2017,cau,1260.0,1,/works/OL19713398W
23,/type/edition,/books/OL26927268M,Owlkids Books,When planet Earth was new,The story of planet Earth s evolution: 4.5 bil...,NaN,"['Origin', 'Life', 'Natural history', 'Evoluti...",2019-05-23T21:54:23.974566,['9781771472036'],2017,onc,NaN,3,/works/OL19714130W
25,/type/edition,/books/OL26927601M,Huang guan wen hua chu ban you xian gong si,"Ren sheng a, zhen de shi mei xiang dao",NaN,NaN,"['Chinese essays', 'Anecdotes']",2019-05-23T22:01:03.251239,['9789573333043'],2017,ch,223.0,1,/works/OL19714463W


In [7]:
df.isna().sum()

record_type              0
book_id                  0
publishers               0
title                    0
description        1208115
first_sentence     1229038
subjects                 0
date_created             0
isbn_13                  0
publish_date             0
publish_country    1197988
number_of_pages     788961
latest_revision          0
work_id                  0
dtype: int64

In [17]:
def clean_publish_date(date):
    """Standardizes publish dates to YYYY-MM-DD format where possible."""
    if pd.isna(date):  
        return None
    
    date = str(date).strip()

    # Try full date parsing (YYYY-MM-DD, YYYY/MM/DD, YYYY.MM.DD, etc.)
    try:
        return parser.parse(date, fuzzy=True).strftime("%Y-%m-%d")
    except:
        pass

    # Match YYYY-MM
    match = re.match(r"(\d{4})[-/.](\d{2})", date)
    if match:
        return f"{match.group(1)}-{match.group(2)}"

    # Match YYYY
    match = re.match(r"(\d{4})$", date)
    if match:
        return match.group(1)

    # Handle noisy data with a valid year
    match = re.search(r"(\d{4})", date)
    if match:
        return match.group(1)

    return None

def extrapolate_missing_dates(df):
    """Fills missing dates using the most common full date in that year."""
    
    # Extract year from cleaned dates
    df["year"] = df["clean_publish_date"].str[:4]

    # Find most common YYYY-MM-DD for each year
    common_dates = (
        df[df["clean_publish_date"].str.len() == 10]  # Keep full YYYY-MM-DD dates
        .groupby("year")["clean_publish_date"]
        .agg(lambda x: Counter(x).most_common(1)[0][0] if len(x) > 0 else None)
        .to_dict()
    )

    # Replace incomplete dates (YYYY or YYYY-MM) with the most common YYYY-MM-DD
    def fill_missing_dates(row):
        if len(str(row["clean_publish_date"])) < 10:  # Not full YYYY-MM-DD
            return common_dates.get(row["year"], None)  # Use most common date
        return row["clean_publish_date"]

    df["final_publish_date"] = df.apply(fill_missing_dates, axis=1)

    return df.drop(columns=["year"])

In [20]:
df["clean_publish_date"] = df["publish_date"].apply(clean_publish_date)
df["year"] = df["clean_publish_date"].astype(str).str[:4].astype(int)
df.head()

,record_type,book_id,publishers,title,description,first_sentence,subjects,date_created,isbn_13,publish_date,publish_country,number_of_pages,latest_revision,work_id,clean_publish_date,year
2,/type/edition,/books/OL26474165M,Harlequin Books,The Kentucky cowboy's baby,Pepper Bourne has big plans for her stepdaddy'...,NaN,"['Fiction', 'Ranches', 'Man-woman relationship...",2018-07-22T05:36:52.580542,['9780373757244'],2016,onc,216.0,3,/works/OL17897021W,2016-02-23,2016
4,/type/edition,/books/OL26869357M,Si chuan wen yi chu ban she,Bai ma ke xin de xing kong,NaN,NaN,"['Zhang pian xiao shuo', 'Er tong wen xue']",2019-04-28T07:08:59.763359,['9787541143113'],2016,cc,259.0,1,/works/OL19649589W,2016-02-23,2016
19,/type/edition,/books/OL26926535M,U Media Global,Bu dong sa jiao de nü ren,NaN,NaN,"['Television mini-series', 'Chinese Television...",2019-05-23T21:39:27.766919,000,2017,cau,1260.0,1,/works/OL19713398W,2017-02-23,2017
23,/type/edition,/books/OL26927268M,Owlkids Books,When planet Earth was new,The story of planet Earth s evolution: 4.5 bil...,NaN,"['Origin', 'Life', 'Natural history', 'Evoluti...",2019-05-23T21:54:23.974566,['9781771472036'],2017,onc,NaN,3,/works/OL19714130W,2017-02-23,2017
25,/type/edition,/books/OL26927601M,Huang guan wen hua chu ban you xian gong si,"Ren sheng a, zhen de shi mei xiang dao",NaN,NaN,"['Chinese essays', 'Anecdotes']",2019-05-23T22:01:03.251239,['9789573333043'],2017,ch,223.0,1,/works/OL19714463W,2017-02-23,2017


In [24]:
df["year"].unique()

array([2016, 2017, 2018, 2020, 2019, 2022, 2035, 2021, 2025, 2023, 2051,
       2050, 2028, 2024, 2075, 2026, 2099, 2034, 2033, 2045, 2030, 2043,
       2079, 2029, 2049, 2027, 2065, 2098, 2039, 2040, 2096, 2036, 2031,
       2044, 2091, 2032, 2081, 2055, 2090, 2097, 2063, 2093, 2074, 2088,
       2060, 2080, 2094, 2076, 2041, 2095, 2047, 2087, 2067, 2077, 2092,
       2059, 2058, 2042, 2048, 2046, 2037])

In [26]:
valid_cols = [
    "book_id", "publishers", "title", "subjects",
    "clean_publish_date", "year", "latest_revision"
]

In [27]:
valid_df = df[df["year"]>2022][valid_cols]
valid_df.head()

,book_id,publishers,title,subjects,clean_publish_date,year,latest_revision
752,/books/OL28802597M,Penguin Publishing Group,Pioneers,['American literature'],2035-02-23,2035,2
1355,/books/OL29579074M,Transworld Publishers Limited,Young Mind,"['Child mental health', 'Child psychology', 'C...",2025-02-23,2025,1
1690,/books/OL33431398M,Taylor & Francis Group,Physical Principles in Nucleic Acid Chemistry,['Nucleic acids'],2023-02-23,2023,1
1691,/books/OL33459375M,Taylor & Francis Group,Modern Sports and Diplomacy,"['Sports, history']",2023-02-23,2023,1
1692,/books/OL33459816M,Elsevier - Health Sciences Division,"Thoracic Anesthesia, an Issue of Anesthesiolog...","['Chest, surgery', 'Anesthesiology']",2051-02-23,2051,1


In [28]:
valid_df.shape

(20454, 7)

In [29]:
valid_df.isna().sum()

book_id               0
publishers            0
title                 0
subjects              0
clean_publish_date    0
year                  0
latest_revision       0
dtype: int64